In [42]:
import numpy as np
import scipy.io as sio
import csv
from nilearn import plotting
import nibabel as nib
import scipy

In [87]:
def get_latent_arr(img_arr, f_lat, m_lat):

    lat_list=[]
    for condition, idx, gender in img_arr:
        if condition!=1: continue
            
        if gender==1: 
            lat_map=f_lat 
        else: 
            lat_map=m_lat
        lat = lat_map[:,1][idx-1][0]
        lat_list.append(lat)

    lat_arr = np.array(lat_list)
    return lat_arr

def get_stim_arr(behavior_list):
    stim_list=[]
    for d in behavior_list:
        mat = scipy.io.loadmat(d)
        condition=mat['designMat'][1]
        imggender=mat['designMat'][2]
        imgidxs=mat['designMat'][3]
        
        for idx in range(len(imgidxs)):
            if condition[idx]==1 or condition[idx]==2:
                stim_list.append([ int(condition[idx]),
                                  int(imgidxs[idx]), 
                                  int(imggender[idx]),
                                ])
    stim_arr=np.array(stim_list)
    return stim_arr

def get_voxel_arr(phase_run_list, tstat_list, mask_name, stim_arr):
    
    mask = nib.load(mask_name)
    mask = mask.get_fdata()
    print('*mask info: ', mask_name, mask.shape, mask[mask==1].shape)
    voxel_list=[]
    for r in phase_run_list:
        for t in tstat_list:
            tstat=nib.load('%s/%s'%(r, t))
            tstat=tstat.get_fdata()
            tstat_masked = tstat[mask==1]
            voxel_list.append(tstat_masked)
            
    voxel_arr = np.array(voxel_list)
    voxel_arr = voxel_arr[stim_arr[:,0]==1]
    return voxel_arr    

In [145]:
SUBJ='sub22'
path_mask = './results/%s/mask/mask1.nii'%SUBJ
path_run = './results/%s/trial_wise/phase1_trialWiseGLM_train_run00.feat'%SUBJ
path_tstat = 'stats/tstat00_mask1.nii.gz'
path_behavior = './data/%s/behavior/data_phase1_run00.mat'%SUBJ

f_lat = np.load('scripts/f_latent.npy', allow_pickle=True)
m_lat = np.load('scripts/m_latent.npy', allow_pickle=True)

run_list=[phase_run.replace('00', str(i)) for i in range(1, 11)]
tstat_list=[tstat.replace('00', str(i)) for i in range(1, 111)]
behavior_list=[behavior.replace('00', str(i)) for i in range(1, 11)]

In [146]:
stim_arr = get_stim_arr(behavior_list)
stim_arr.shape

(1100, 3)

In [147]:
voxel_arr = get_voxel_arr(run_list, tstat_list, path_mask, stim_arr)
voxel_arr.shape

*mask info:  ./results/sub22/mask/mask1.nii (91, 109, 91) (27347,)


(1000, 27347)

In [148]:
latent_arr= get_latent_arr(stim_arr, f_lat, m_lat)
latent_arr.shape

(1000, 100)

In [144]:
self, others = regression_get_corr(voxel_arr, latent_arr)
np.mean(self), np.mean(others)

(900, 17540) (900, 100)
(100, 17540) (100, 100)
0.1871607667985445 0.15205739380074326


(0.1871607667985445, 0.15205739380074326)

In [149]:
self, others = regression_get_corr(voxel_arr, latent_arr)
np.mean(self), np.mean(others)

(900, 27347) (900, 100)
(100, 27347) (100, 100)
0.19349444646339553 0.14960529096599537


(0.19349444646339553, 0.14960529096599537)

In [118]:
import sklearn

def get_corr(y_test, y_pred):

    corr=[]
    self_sum, other_sum = 0, 0
    self_n, other_n = 0, 0
    for ti, y_t in enumerate(y_test):
        corr_row=[]
        for pi, y_p in enumerate(y_pred):
            co = np.corrcoef(y_t, y_p)[0][1]
            corr_row.append(co)
            if ti==pi: 
                #print(ti, pi, co)
                self_sum +=co
                self_n+=1
            else:
                other_sum +=co
                other_n+=1
        corr.append(corr_row)  
    corr = np.array(corr)
    self_corr = self_sum/self_n
    other_corr = other_sum/other_n
    
    return self_corr, other_corr


def regression_get_corr(act_arr, lat_arr):

    selfs=[]
    others=[]

    for n_regression in range(10):
        mask_train = []
        mask_test = []
        for i in range(10):
            if i==n_regression: 
                w_train = [False for i in range(100)]
                w_test = [True for i in range(100)]
            else: 
                w_train = [True for i in range(100)]
                w_test = [False for i in range(100)]
            mask_train.extend(w_train)
            mask_test.extend(w_test)

        x_train, y_train = act_arr[mask_train], lat_arr[mask_train]
        print(x_train.shape, y_train.shape)
        x_test, y_test = act_arr[mask_test], lat_arr[mask_test]
        print(x_test.shape, y_test.shape)

        line_fitter = sklearn.linear_model.LinearRegression()
        line_fitter.fit(x_train, y_train)
        y_pred = line_fitter.predict(x_test)


        self_corr, other_corr = get_corr(y_test, y_pred)

        print(self_corr, other_corr)
        selfs.append(self_corr)
        others.append(other_corr)
        
        return selfs, others